In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense, AveragePooling2D
from tensorflow.keras.layers import ZeroPadding2D, add, Input, Activation
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import wandb
from wandb.keras import WandbCallback

In [2]:
((trainX, trainY), (testX, testY)) = cifar10.load_data()

trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

In [3]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

labelNames = ["airplane", "automobile", "bird", "cat", "deer",
"dog", "frog", "horse", "ship", "truck"]

In [4]:
sweep_config = {

     'method': 'grid',

    'parameters': {
        'learning_rate': 
         {'values': [1e-2,1e-3]},
        
        'reg': 
            {'values': [0.001, 0.005, 0.009]
                   
                   
                },
        
        'decay': 
            {'values': [1e-1,1e-2,1e-3,1e-4]
                   
                   
                }
     
    
        }

    }
sweep_id = wandb.sweep(sweep_config, project= "ResNet_Cifar10_HyperParam_Adam") 

Create sweep with ID: mcomexkl
Sweep URL: https://app.wandb.ai/roastedkernel/ResNet_Cifar10_HyperParam_Adam/sweeps/mcomexkl


In [5]:
def residual_module(data, K, stride, chanDim,
                    reg=0.0001, bnEps=2e-5, bnMom=0.9, reduce=False):
    
    shortcut = data
    
    bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps,
            momentum=bnMom)(data)
    act1 = Activation("relu")(bn1)
    
    conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False,
                kernel_regularizer=l2(reg))(act1)
    
    
    bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps,
                momentum=bnMom)(conv1)
    act2 = Activation("relu")(bn2)
    conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride,
                padding="same", use_bias=False,
    kernel_regularizer=l2(reg))(act2)
    
    
    bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps,
                momentum=bnMom)(conv2)
    act3 = Activation("relu")(bn3)
    conv3 = Conv2D(K, (1, 1), use_bias=False,
                kernel_regularizer=l2(reg))(act3)
    
    
    
    if reduce:
        shortcut = Conv2D(K, (1, 1), strides=stride,
            use_bias=False, kernel_regularizer=l2(reg))(act1)
    
    x = add([conv3, shortcut])
    
    
    
    return x






def build_Resnet(width, height, depth, classes, stages, filters,
                reg=0.0001, bnEps=2e-5, bnMom=0.9):
    
    
    inputShape = (height, width, depth)
    chanDim = -1
    inputs = Input(shape=inputShape)
    x = BatchNormalization(axis=chanDim, epsilon=bnEps,
                            momentum=bnMom)(inputs)
    x = Conv2D(filters[0], (3, 3), use_bias=False,padding="same", kernel_regularizer=l2(reg))(x)
    
    
    
    for i in range(0, len(stages)):

        stride = (1, 1) if i == 0 else (2, 2)
        
        x = residual_module(x, filters[i + 1], stride,
                chanDim, reduce=True, bnEps=bnEps, bnMom=bnMom)
        
        for j in range(0, stages[i] - 1):
            
            x = residual_module(x, filters[i + 1],
                    (1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)
            
    x = BatchNormalization(axis=chanDim, epsilon=bnEps,
                            momentum=bnMom)(x)
    x = Activation("relu")(x)
    x = AveragePooling2D((8, 8))(x) 
    x = Flatten()(x)
    x = Dense(classes, kernel_regularizer=l2(reg))(x)
    x = Activation("softmax")(x)
    
    model = Model(inputs, x, name="resnet")
    
    
    
    return model


In [6]:
def train():
    wandb.init()
    configs = {'learning_rate': 1e-2,'reg': 0.005, 'decay': 1e-2 }
    config = wandb.config
    config.epochs = 100
    
    
    
    model = build_Resnet(32, 32, 3, 10, (9, 9, 9),
                                (64, 64, 128, 256), reg=wandb.config.reg)
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

    metric = "val_accuracy"

    checkpoint = ModelCheckpoint("ResNet_Cifar10.h5", monitor= metric, verbose=1, save_best_only=True,
                             save_weights_only=False, mode='auto')
    
    
    #n_epochs = 100
    batch = 512
    s = 100 * len(trainX) // batch # number of steps in 20 epochs (batch size = 32)
    learning_rate0 = keras.optimizers.schedules.ExponentialDecay(wandb.config.learning_rate, s, wandb.config.decay)

    opt = Adam(learning_rate0,amsgrad=True)
    #opt = opt = SGD(lr= wandb.config.learning_rate ,decay = wandb.config.decay ,  momentum= 0.9, nesterov=True)
    
    
    model.compile(optimizer=opt,
              loss="categorical_crossentropy",
              metrics=['accuracy'])
    
    wandb_callback = WandbCallback(data_type= "image", validation_data=(testX, testY)
                                 , labels = labelNames)

    history = model.fit(trainX, trainY, epochs=config.epochs, 
                    validation_data=(testX, testY), use_multiprocessing=True,
                   callbacks=[es,checkpoint,wandb_callback], batch_size=batch,verbose=10)


In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: i6belcys with config:
	decay: 0.1
	learning_rate: 0.01
	reg: 0.001
wandb: Agent Started Run: i6belcys


Epoch 1/100

Epoch 00001: val_accuracy improved from -inf to 0.43640, saving model to ResNet_Cifar10.h5
Epoch 2/100

Epoch 00002: val_accuracy improved from 0.43640 to 0.52510, saving model to ResNet_Cifar10.h5
Epoch 3/100

Epoch 00003: val_accuracy improved from 0.52510 to 0.56440, saving model to ResNet_Cifar10.h5
Epoch 4/100

Epoch 00004: val_accuracy improved from 0.56440 to 0.65980, saving model to ResNet_Cifar10.h5
Epoch 5/100

Epoch 00005: val_accuracy did not improve from 0.65980
Epoch 6/100

Epoch 00006: val_accuracy improved from 0.65980 to 0.67310, saving model to ResNet_Cifar10.h5
Epoch 7/100

Epoch 00007: val_accuracy improved from 0.67310 to 0.71430, saving model to ResNet_Cifar10.h5
Epoch 8/100

Epoch 00008: val_accuracy improved from 0.71430 to 0.73670, saving model to ResNet_Cifar10.h5
Epoch 9/100

Epoch 00009: val_accuracy did not improve from 0.73670
Epoch 10/100

Epoch 00010: val_accuracy improved from 0.73670 to 0.74570, saving model to ResNet_Cifar10.h5
Epoch 11/1